In [28]:
!pip uninstall opencv-python opencv-contrib-python -y
!pip install opencv-contrib-python==4.6.0.66 --quiet
!pip install -U ultralytics filterpy --quiet


Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.3 MB/s eta 0:00:00


In [26]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter
import time
import random
from google.colab import files

In [3]:
uploaded = files.upload()
input_path = list(uploaded.keys())[0]


Saving person4.mp4 to person4.mp4


In [22]:
model = YOLO('yolov8m.pt')

In [29]:

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = 'tracked_hybrid.mp4'
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

frame_id = 0
names = model.names
known_ids = set()
trackers = dict()
track_history = dict()
missing_counter = dict()
last_position = dict()
kalman_filters = dict()

allowed_classes = [0, 2, 3]  # Person, Car, Motorcycle
MAX_MISSING = 120
MAX_SPEED = 120
MIN_OVERLAP_IOU = 0.005

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def init_kalman():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    kf.P *= 500
    kf.R *= 1.5
    kf.Q *= 0.003
    return kf

def create_tracker(tracker_type):
    try:
        if tracker_type == 'CSRT':
            return cv2.legacy.TrackerCSRT_create()
        elif tracker_type == 'KCF':
            return cv2.legacy.TrackerKCF_create()
        elif tracker_type == 'MOSSE':
            return cv2.legacy.TrackerMOSSE_create()
        else:
            return None
    except AttributeError as e:
        print(f"Error: Tracker {tracker_type} not available. Falling back to KCF.")
        return cv2.legacy.TrackerKCF_create()  # در صورت عدم دسترسی به MOSSE، به KCF بازمی‌گردد

track_id_counter = 0

while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    # تشخیص با YOLO و ردیابی با ByteTrack
    results = model.track(source=frame, persist=True, verbose=False, tracker='bytetrack.yaml', imgsz=1280, conf=0.01, iou=0.3)

    current_ids = set()
    filtered_boxes = []

    # به‌روزرسانی ردیاب‌های OpenCV
    for track_id in list(trackers.keys()):
        if trackers[track_id]['type'] != 'ByteTrack':
            tracker = trackers[track_id]['tracker']
            success, bbox = tracker.update(frame)
            if success:
                x1, y1, w, h = map(int, bbox)
                x2, y2 = x1 + w, y1 + h
                cx = (x1 + x2) / 2
                cy = (y1 + y2) / 2

                kalman_filters[track_id].predict()
                kalman_filters[track_id].update(np.array([[cx], [cy]]))
                cx, cy = kalman_filters[track_id].x[0][0], kalman_filters[track_id].x[1][0]

                if track_id in track_history:
                    prev_cx, prev_cy = track_history[track_id]
                    dx = abs(cx - prev_cx)
                    dy = abs(cy - prev_cy)
                    speed = (dx**2 + dy**2)**0.5
                else:
                    speed = 0

                track_history[track_id] = (cx, cy)
                current_ids.add(track_id)

                if track_id in known_ids and speed < MAX_SPEED:
                    if track_id in last_position:
                        iou = calculate_iou([x1, y1, x2, y2], last_position[track_id])
                        if iou >= MIN_OVERLAP_IOU:
                            filtered_boxes.append({'id': track_id, 'xyxy': [x1, y1, x2, y2], 'cls': trackers[track_id]['cls']})
                            last_position[track_id] = [x1, y1, x2, y2]
                    else:
                        last_position[track_id] = [x1, y1, x2, y2]
                        filtered_boxes.append({'id': track_id, 'xyxy': [x1, y1, x2, y2], 'cls': trackers[track_id]['cls']})
            else:
                missing_counter[track_id] = missing_counter.get(track_id, 0) + 1
                if missing_counter[track_id] > MAX_MISSING:
                    known_ids.discard(track_id)
                    trackers.pop(track_id, None)
                    kalman_filters.pop(track_id, None)

    # پردازش نتایج ByteTrack و افزودن اشیای جدید
    for box in results[0].boxes:
        if box.id is None:
            continue
        cls_id = int(box.cls.item())
        if cls_id not in allowed_classes:
            continue
        track_id = int(box.id.item())
        xyxy = box.xyxy[0].cpu().numpy().tolist()
        x1, y1, x2, y2 = map(int, xyxy)
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2

        # انتخاب ردیاب: 70% ByteTrack، 15% CSRT، 10% KCF، 5% MOSSE
        if track_id not in trackers:
            tracker_type = np.random.choice(['ByteTrack', 'CSRT', 'KCF', 'MOSSE'], p=[0.7, 0.15, 0.1, 0.05])
            if tracker_type != 'ByteTrack':
                tracker = create_tracker(tracker_type)
                if tracker is not None:
                    tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                    trackers[track_id] = {'tracker': tracker, 'type': tracker_type, 'cls': cls_id}
                else:
                    continue  # در صورت عدم دسترسی به ردیاب، نادیده گرفته می‌شود
            else:
                trackers[track_id] = {'type': 'ByteTrack', 'cls': cls_id}
            if frame_id <= 50:  # فقط تا فریم 50 اشیای جدید به known_ids اضافه می‌شوند
                known_ids.add(track_id)
            kalman_filters[track_id] = init_kalman()
            kalman_filters[track_id].x[:2] = np.array([[cx], [cy]])

        if trackers[track_id]['type'] == 'ByteTrack':
            kalman_filters[track_id].predict()
            kalman_filters[track_id].update(np.array([[cx], [cy]]))
            cx, cy = kalman_filters[track_id].x[0][0], kalman_filters[track_id].x[1][0]

            if track_id in track_history:
                prev_cx, prev_cy = track_history[track_id]
                dx = abs(cx - prev_cx)
                dy = abs(cy - prev_cy)
                speed = (dx**2 + dy**2)**0.5
            else:
                speed = 0

            track_history[track_id] = (cx, cy)
            current_ids.add(track_id)

            if track_id in known_ids and speed < MAX_SPEED:
                if track_id in last_position:
                    iou = calculate_iou(xyxy, last_position[track_id])
                    if iou >= MIN_OVERLAP_IOU:
                        filtered_boxes.append({'id': track_id, 'xyxy': xyxy, 'cls': cls_id})
                        last_position[track_id] = xyxy
                else:
                    last_position[track_id] = xyxy
                    filtered_boxes.append({'id': track_id, 'xyxy': xyxy, 'cls': cls_id})

    # مدیریت اشیای گمشده
    for track_id in list(known_ids):
        if track_id not in current_ids:
            missing_counter[track_id] = missing_counter.get(track_id, 0) + 1
            if missing_counter[track_id] > MAX_MISSING:
                known_ids.discard(track_id)
                trackers.pop(track_id, None)
                kalman_filters.pop(track_id, None)
        else:
            missing_counter[track_id] = 0

    # ترسیم جعبه‌ها و برچسب‌ها
    annotated_frame = frame.copy()
    for box in filtered_boxes:
        x1, y1, x2, y2 = map(int, box['xyxy'])
        cls_id = box['cls']
        track_id = box['id']
        label = f"{names[cls_id]} ID:{track_id} ({trackers[track_id]['type']})"
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(annotated_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # نمایش FPS
    end_time = time.time()
    frame_time = end_time - start_time
    current_fps = 1.0 / frame_time if frame_time > 0 else 0
    cv2.putText(annotated_frame, f"FPS: {current_fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(annotated_frame)
    frame_id += 1

cap.release()
out.release()


Error: Tracker KCF not available. Falling back to KCF.


AttributeError: module 'cv2.legacy' has no attribute 'TrackerKCF_create'

In [20]:
files.download(out_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>